# I saved pre processed text in csv file m using that dataset

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from nltk import WordNetLemmatizer
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Bidirectional, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.optimizers import Adam

In [3]:
urdu_dataset = pd.read_csv("Analysed_Urdu_Tweets.csv",index_col=False)
urdu_dataset.drop(urdu_dataset.columns[0],axis=1,inplace=True)
urdu_dataset.head()


,Tweet,Class
0,ایٹم بم بنایا ھے بھائی ایٹم بمب کوٹ لکھپت اتفا...,2
1,چندے انقلاب عمران خان وزیر اعظم بن سکتے,0
2,ٹویٹر خیال,1
3,سرچ انجن گوگل نائب صدر فضا فٹ بلندی چھلانگ عال...,2
4,اسکی لہریں یار أ,2


In [4]:

## WORDS in dataset
tokenizer = Tokenizer()
tokenizer.fit_on_texts(urdu_dataset['Tweet'])
sequences = tokenizer.texts_to_sequences(urdu_dataset['Tweet'])

max_sequence_length = max(len(x) for x in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')
## Split the dataset
# Splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, urdu_dataset['Class'], test_size=0.25, random_state=42)

print("Training set shape:",X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (749, 31)
Testing set shape: (250, 31)


# Models

In [9]:
import gensim.downloader as api
import tensorflow_hub as hub
word2vec_model = api.load("word2vec-google-news-300")
glove_model = api.load("glove-twitter-200")
fasttext_model = api.load("fasttext-wiki-news-subwords-300")
elmo = hub.load("https://tfhub.dev/google/elmo/2")

In [10]:
max_length = max([len(seq) for seq in X_train])
X_train_padded = pad_sequences(X_train, maxlen=max_length)
X_test_padded = pad_sequences(X_test, maxlen=max_length)

In [11]:
# word2vec_weights = create_embedding_matrix(word2vec_model, tokenizer)

vocab_size = len(tokenizer.word_index) + 1
embed_matrix = np.zeros((vocab_size,word2vec_model.vector_size))

for w,i in tokenizer.word_index.items():
    if w in word2vec_model:
        embed_matrix[i] = word2vec_model[w]
word2vec_weights = embed_matrix

In [43]:
# LSTM -> 3,0.7 ->0.66
# nested for for dropout and num_layers

num_layers =3
dropout_rate=0.7

embedding_dim = 128

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(dropout_rate))
model.add(LSTM(units=64))
model.add(Dropout(dropout_rate))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [44]:
model.fit(X_train, y_train, batch_size=32, epochs=8, validation_split=0.25)

Epoch 1/8
18/18 [==============================] - 15s 195ms/step - loss: 0.0000e+00 - accuracy: 0.4171 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 2/8
18/18 [==============================] - 2s 91ms/step - loss: 0.0000e+00 - accuracy: 0.4866 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 3/8
18/18 [==============================] - 1s 72ms/step - loss: 0.0000e+00 - accuracy: 0.4225 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 4/8
18/18 [==============================] - 1s 73ms/step - loss: 0.0000e+00 - accuracy: 0.3583 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 5/8
18/18 [==============================] - 1s 78ms/step - loss: 0.0000e+00 - accuracy: 0.3815 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 6/8
18/18 [==============================] - 2s 86ms/step - loss: 0.0000e+00 - accuracy: 0.4349 - val_loss: 0.0000e+00 - val_accuracy: 0.5160
Epoch 7/8
18/18 [==============================] - 1s 74ms/step - loss: 0.0000e+00 - accuracy: 0.4260 - val_lo

In [45]:
model_eval = dict()
from sklearn.metrics import precision_score, recall_score, f1_score

In [46]:
y_pred = model.predict(X_test_padded)
y_pred = (y_pred > 0.5).astype(int)
precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
loss, accuracy = model.evaluate(X_test, y_test)

model_eval['LSTM(no embedding)'] = {'Precision':precision,'Recall':recall,'F1 score':f1,'accuracy':accuracy}

8/8 [==============================] - 3s 11ms/step


In [47]:
model_eval

{'LSTM(no embedding)': {'Precision': 0.16266666666666665,
  'Recall': 0.3333333333333333,
  'F1 score': 0.2186379928315412}}

## word2vec

In [48]:
model = Sequential()
model.add(Embedding(input_dim=len(word2vec_weights), output_dim=300,weights=[word2vec_weights], input_length=max_length))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(dropout_rate))
model.add(LSTM(units=64))
model.add(Dropout(dropout_rate))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [49]:
model.fit(X_train_padded, y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
10/10 [==============================] - 24s 261ms/step - loss: 0.0000e+00 - accuracy: 0.3205 - val_loss: 0.0000e+00 - val_accuracy: 0.5200
Epoch 2/10
10/10 [==============================] - 1s 118ms/step - loss: 0.0000e+00 - accuracy: 0.4324 - val_loss: 0.0000e+00 - val_accuracy: 0.5200
Epoch 3/10
10/10 [==============================] - 1s 120ms/step - loss: 0.0000e+00 - accuracy: 0.4140 - val_loss: 0.0000e+00 - val_accuracy: 0.5200
Epoch 4/10
10/10 [==============================] - 1s 118ms/step - loss: 0.0000e+00 - accuracy: 0.3873 - val_loss: 0.0000e+00 - val_accuracy: 0.5200
Epoch 5/10
10/10 [==============================] - 1s 140ms/step - loss: 0.0000e+00 - accuracy: 0.4190 - val_loss: 0.0000e+00 - val_accuracy: 0.5200
Epoch 6/10
10/10 [==============================] - 2s 169ms/step - loss: 0.0000e+00 - accuracy: 0.4391 - val_loss: 0.0000e+00 - val_accuracy: 0.5200
Epoch 7/10
10/10 [==============================] - 1s 145ms/step - loss: 0.0000e+00 - accuracy: 0.

In [50]:
y_pred = model.predict(X_test_padded)
y_pred = (y_pred > 0.5).astype(int)
precision2 = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall2 = recall_score(y_test, y_pred, average='macro', zero_division=0)
f12 = f1_score(y_test, y_pred, average='macro', zero_division=0)
loss, accuracy2 = model.evaluate(X_test, y_test)

8/8 [==============================] - 1s 17ms/step


In [51]:
model_eval['LSTM(embedding: word2vec)'] = {'Precision':precision2,'Recall':recall2,'F1 score':f12,'accuracy':accuracy2}

In [52]:
table = pd.DataFrame.from_dict(model_eval, orient='index')
print(table)

                          Precision    Recall  F1 score
LSTM(no embedding)         0.162667  0.333333  0.218638
GRU(embedding: word2vec)   0.162667  0.333333  0.218638


## glove

embed matrix

In [57]:
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in glove_vectors.key_to_index:
        embedding_matrix[i] = glove_vectors[word]

model 

In [58]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(LSTM(units=128))
model.add(Dense(units=3, activation='softmax')) #3 classes

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [59]:
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
12/12 [==============================] - 10s 259ms/step - loss: 0.9931 - accuracy: 0.4887 - val_loss: 0.8166 - val_accuracy: 0.4880
Epoch 2/10
12/12 [==============================] - 1s 80ms/step - loss: 0.8053 - accuracy: 0.5180 - val_loss: 0.8075 - val_accuracy: 0.4880
Epoch 3/10
12/12 [==============================] - 1s 83ms/step - loss: 0.7776 - accuracy: 0.4740 - val_loss: 0.7957 - val_accuracy: 0.4880
Epoch 4/10
12/12 [==============================] - 1s 85ms/step - loss: 0.7767 - accuracy: 0.5033 - val_loss: 0.7895 - val_accuracy: 0.4880
Epoch 5/10
12/12 [==============================] - 1s 78ms/step - loss: 0.7742 - accuracy: 0.4993 - val_loss: 0.8077 - val_accuracy: 0.4880
Epoch 6/10
12/12 [==============================] - 1s 87ms/step - loss: 0.7789 - accuracy: 0.4993 - val_loss: 0.7896 - val_accuracy: 0.4880
Epoch 7/10
12/12 [==============================] - 1s 104ms/step - loss: 0.7729 - accuracy: 0.5113 - val_loss: 0.7945 - val_accuracy: 0.4880
Epoch 8/10

In [60]:
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

8/8 [==============================] - 1s 26ms/step


Evaluate

In [61]:
loss, accuracy3 = model.evaluate(X_test, y_test)
precision3 = precision_score(y_test, y_pred_labels, average='macro')
recall3 = recall_score(y_test, y_pred_labels, average='macro')
f13 = f1_score(y_test, y_pred_labels, average='macro')
model_eval['LSTM(embedding: GLOVE)'] = {'Precision':precision3,'Recall':recall3,'F1 score':f13,'accuracy':accuracy3}

8/8 [==============================] - 0s 26ms/step - loss: 0.7900 - accuracy: 0.4880


d:\Python64\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Fast Text

In [64]:
fasttext_model = api.load("fasttext-wiki-news-subwords-300")

KeyboardInterrupt: 

Embed matrix

In [65]:
embed_matrix = np.zeros((vocab_size,fasttext_model.vector_size))

for w,i in tokenizer.word_index.items():
    if w in fasttext_model:
        embed_matrix[i] = word2vec_model[w]
fasttext_weights = embed_matrix    


NameError: name 'fasttext_model' is not defined

Model

In [66]:
input_dim = len(fasttext_weights)

model=Sequential()
model.add(Embedding(input_dim=input_dim, output_dim=output_dim, weights=[embedding_matrix], input_length=input_length, trainable=False))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


NameError: name 'fasttext_weights' is not defined

In [ ]:
model.fit(X_train_padded, y_train, epochs=10, batch_size=64, validation_split=0.2)


In [ ]:
y_pred = model.predict(X_test_padded)
y_pred = (y_pred > 0.5).astype(int)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
loss, accuracy4 = model.evaluate(X_test, y_test)
model_eval['LSTM(embedding: fast-text)'] = {'Precision':precision2,'Recall':recall2,'F1 score':f12,'accuracy':accuracy4}

Elmo

In [ ]:
elmo = hub.load("https://tfhub.dev/google/elmo/2")

In [63]:
table = pd.DataFrame.from_dict(model_eval, orient='index')
print(table)

                          Precision    Recall  F1 score  accuracy
LSTM(no embedding)         0.162667  0.333333  0.218638       NaN
GRU(embedding: word2vec)   0.162667  0.333333  0.218638       NaN
LSTM(embedding: GLOVE)     0.162667  0.333333  0.218638     0.488
